In [1]:
import os
import io
import datetime
import logging
import sys

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize 

import civis
import civis.io
from civis.futures import CivisFuture

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputRegressor

from civis.ml import ModelPipeline
from muffnn import MLPClassifier, MLPRegressor

import uuid
import json
import math
from pprint import pprint
import tempfile
import concurrent.futures
from concurrent.futures import wait
from collections import namedtuple

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Set Parameter
DATABASE = 'Bernie 2020'
PRIMARY_KEY = 'person_id'
DV_TABLE = 'bernie_nmarchio2.spoke_dvs'

DV_LIST = ['spoke_support_1box', 'spoke_persuasion_1plus', 'spoke_persuasion_1minus']

MODELING_FRAME = 'bernie_data_commons.phoenix_modeling_frame'
EXCLUSION_COLUMNS = ['jsonid']

SCHEMA = 'bernie_nmarchio2'
PREFIX = 'scored'


In [3]:
datestamp = '{:%Y%m%d}'.format(datetime.date.today())
score_table = f'{SCHEMA}.{PREFIX}_output_{datestamp}'

In [4]:
# Count positive and negative classes
dv_sql_targets = "\n,".join(["sum({dv}) as {dv}".format(dv=i) for i in DV_LIST])
sql_collapse_targets = f"""select {dv_sql_targets} from {DV_TABLE};"""
sql_count_targets = civis.io.read_civis_sql(sql_collapse_targets, DATABASE)

dv_sql_zeroes = "\n,".join(["sum(case when {dv} = 0 then 1 end) as {dv}".format(dv=i) for i in DV_LIST])
sql_collapse_zeroes = f"""select {dv_sql_zeroes} from {DV_TABLE};"""
sql_count_zeroes = civis.io.read_civis_sql(sql_collapse_zeroes, DATABASE)


In [5]:
# Share of negatives to sample based on number of positives (to avoid class imbalance problems)
sample_share = []
for i in range(len(DV_LIST)):
    u = round((int(sql_count_targets[1][i])*2)/(int(sql_count_zeroes[1][i])),5)
    sample_share.append(u)

In [6]:
# Create training views
for i in range(len(DV_LIST)):
    dv_item = DV_LIST[i]
    random_sample = sample_share[i]
    training_sql = f"""DROP VIEW IF EXISTS {SCHEMA}.{PREFIX}_training_{i} CASCADE;
    CREATE VIEW {SCHEMA}.{PREFIX}_training_{i} AS 
    (select * from (
    (select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 1) 
    union all 
    (select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 0 and random() < {random_sample}))
    left join {MODELING_FRAME} using({PRIMARY_KEY}));"""
    create_training_sql = civis.io.query_civis(training_sql, database=DATABASE)
    create_training_sql.result().state
 

In [7]:
# Train models
train_list = []
model_list = []

for index, dv in enumerate(DV_LIST):
    print('TRAINING >>> {}'.format(dv))
    
    exc_list = DV_LIST.copy()
    exc_list.remove(dv)
    
    assert dv not in exc_list 
    
    name = f"""{dv}_{datestamp}"""
    model = ModelPipeline(model='sparse_logistic',
                          dependent_variable=dv,
                          primary_key=PRIMARY_KEY,
                          excluded_columns=EXCLUSION_COLUMNS,
                          calibration='sigmoid',
                          model_name=name,
                          memory_requested=12000)
    
    where_string = '{} is not null'.format(dv)

    train = model.train(table_name=f"""{SCHEMA}.{PREFIX}_training_{index}""", 
                        database_name=DATABASE,
                        sql_where=where_string#,
                        #fit_params={'sample_weight': WEIGHT_VAR}
                       )
    
    model_list.append(model)
    train_list.append(train)    


TRAINING >>> spoke_support_1box
TRAINING >>> spoke_persuasion_1plus
TRAINING >>> spoke_persuasion_1minus


In [8]:
# Get output of successful models
model_output = model_list
train_output = train_list

jobs_list = []
for t in train_output: 
    try:
        if len(t.metadata['output']) > 0:  
            jobs_list.append(t)
            print('Job success')
    except:
        print('Job failure')
        pass

model_output, train_ouput = zip(*((model, train) for model, train in zip(model_output, train_output) if train in jobs_list))
model_output = list(model_output)
train_output = list(train_output)

Job success
Job success
Job success


In [9]:
# Generate validation metrics
metrics_list = []

for a, b in enumerate(train_output):
    metric = {'job_id':b.job_id,
              'run_id':b.run_id,
              'dv': ''.join(b.metadata['run']['configuration']['data']['y']),
              'model': b.metadata['model']['model'],
              'time_of_train_run': b.metadata['run']['time_of_run'],
              'n_rows': b.metadata['data']['n_rows'],
              'n_features': b.metadata['data']['n_cols'],
              'auc': b.metadata['metrics']['roc_auc'],
              'deciles': b.metadata['metrics']['deciles'],
              'confusion_matrix': b.metadata['metrics']['confusion_matrix'],
              'accuracy': b.metadata['metrics']['accuracy'],
              'p_correct': b.metadata['metrics']['p_correct'],
              'pop_incidence_true': b.metadata['metrics']['pop_incidence_true']
             }
    metrics_list.append(metric)
    
metric_order = (['job_id', 'run_id', 'dv', 'model', 'time_of_train_run', 'n_rows', 'n_features',
                 'auc', 'deciles', 'confusion_matrix', 'accuracy', 'p_correct','pop_incidence_true'])

validation_df = pd.DataFrame.from_records(metrics_list, columns=metric_order, index='run_id')
validation_df

,job_id,dv,model,time_of_train_run,n_rows,n_features,auc,deciles,confusion_matrix,accuracy,p_correct,pop_incidence_true
run_id,,,,,,,,,,,,
189716219,52069719,spoke_support_1box,sparse_logistic,2019-12-10T17:08:23Z,155925,44,0.731163,"[0.08119548486403284, 0.13486821009427308, 0.1...","[[90571, 13329], [32204, 19821]]",0.707981,"[0.8717131857555341, 0.38098990869774146]","[0.6663459996793331, 0.333654000320667]"
189716240,52069728,spoke_persuasion_1plus,sparse_logistic,2019-12-10T17:07:49Z,8673,44,0.608623,"[0.18223760092272204, 0.23617511520737328, 0.2...","[[5739, 68], [2791, 75]]",0.670356,"[0.9882899948338213, 0.026168876482903]","[0.6695491756024443, 0.33045082439755563]"
189716247,52069736,spoke_persuasion_1minus,sparse_logistic,2019-12-10T17:07:48Z,2272,44,0.709138,"[0.09251101321585903, 0.15418502202643172, 0.1...","[[1345, 170], [544, 213]]",0.685739,"[0.8877887788778878, 0.2813738441215324]","[0.6668133802816901, 0.3331866197183099]"


In [10]:
# Write validation metrics to Redshift
create_validation_table = civis.io.dataframe_to_civis(df=validation_df,
                                                 database=DATABASE, 
                                                 table= f'{SCHEMA}.model_validation_{datestamp}', 
                                                 existing_table_rows='drop')

In [11]:
# Score the voterfile
scores_list = []
for m,t in zip(model_output, train_output):
    DV_NAME = ''.join(t.metadata['run']['configuration']['data']['y'])
    print(DV_NAME)
    SCORES_TABLE = f'{SCHEMA}.{PREFIX}_{DV_NAME}_{datestamp}'
    scores_list.append(SCORES_TABLE)
    scores = m.predict(primary_key=PRIMARY_KEY,
                       database_name=DATABASE, 
                       table_name=MODELING_FRAME,
                       if_exists='drop',
                       output_table=SCORES_TABLE,
                       disk_space=20)
scores.result()

spoke_support_1box
spoke_persuasion_1plus
spoke_persuasion_1minus


{'container_id': 52070458,
 'error': None,
 'finished_at': '2019-12-10T17:30:29.000Z',
 'id': 189717357,
 'is_cancel_requested': False,
 'started_at': '2019-12-10T17:09:35.000Z',
 'state': 'succeeded'}

In [12]:
# Grant team on tables
grant_sql = "".join(["GRANT SELECT ON {tbl} TO GROUP bernie_data;".format(tbl=i) for i in scores_list])
grant_statement = f"""
GRANT ALL ON SCHEMA {SCHEMA} TO GROUP bernie_data;
{grant_sql}
"""
grant_team = civis.io.query_civis(sql=grant_statement, database=DATABASE)
grant_team.result().state

'succeeded'

In [16]:
# Drop training views
drop_statement = []
for i in range(len(DV_LIST)):
    drop_sql = f"drop view if exists {SCHEMA}.{PREFIX}_training_{i} CASCADE;"
    drop_statement.append(drop_sql)
drop_sql = "".join(["{tbl}".format(tbl=i) for i in drop_statement])
drop_views = civis.io.query_civis(sql=drop_sql, database=DATABASE)
drop_views.result().state

'succeeded'

In [85]:
# Join everything together and decile / drop redundant tables and views
view_list = []
table_list = []
for i in range(len(DV_LIST)):
    view = f"{SCHEMA}.{PREFIX}_training_{i}"
    view_list.append(view)
    table = f"{SCHEMA}.{PREFIX}_{DV_LIST[i]}_{datestamp}"
    table_list.append(table)

drop_view = "".join(["drop view if exists {view} CASCADE;".format(view=v) for v in view_list])
drop_table = "".join(["drop table if exists {tbl} CASCADE;".format(tbl=t) for t in table_list])
dv_strings = "\n,".join(["{dv_score}_1 as {dv_score}".format(dv_score=dv) for dv in DV_LIST])
dv_tiles = "\n,".join(["NTILE(100) OVER (ORDER BY {dv_tile}_1) AS {dv_tile}_100".format(dv_tile=dv) for dv in DV_LIST])
join_table = []
if len(table_list) > 1:
    for i in table_list[1:]:
        j = str(' left join '+f'{i}'+f' using({PRIMARY_KEY}) ')
        join_table.append(j)
        #dv_strings = "\nleft join ".join(["{dv_score}".format(table=tbl) for tbl in table_list[i])


In [88]:
output_table = f"""DROP TABLE IF EXISTS {SCHEMA}.{PREFIX}_output_{datestamp};
CREATE TABLE {SCHEMA}.{PREFIX}_output_{datestamp}
  DISTSTYLE KEY
  DISTKEY ({PRIMARY_KEY})
  SORTKEY ({PRIMARY_KEY})
  AS ("""+'select '+ f"{PRIMARY_KEY} \n," + dv_strings + "\n," + dv_tiles + ' from '+ ''.join(table_list[0]) + ''.join(join_table) +');'  



In [89]:
print(output_table)

DROP TABLE IF EXISTS bernie_nmarchio2.scored_output_20191210;
CREATE TABLE bernie_nmarchio2.scored_output_20191210
  DISTSTYLE KEY
  DISTKEY (person_id)
  SORTKEY (person_id)
  AS (select person_id 
,spoke_support_1box_1 as spoke_support_1box
,spoke_persuasion_1plus_1 as spoke_persuasion_1plus
,spoke_persuasion_1minus_1 as spoke_persuasion_1minus
,NTILE(100) OVER (ORDER BY spoke_support_1box_1) AS spoke_support_1box_100
,NTILE(100) OVER (ORDER BY spoke_persuasion_1plus_1) AS spoke_persuasion_1plus_100
,NTILE(100) OVER (ORDER BY spoke_persuasion_1minus_1) AS spoke_persuasion_1minus_100 from bernie_nmarchio2.scored_spoke_support_1box_20191210 left join bernie_nmarchio2.scored_spoke_persuasion_1plus_20191210 using(person_id)  left join bernie_nmarchio2.scored_spoke_persuasion_1minus_20191210 using(person_id) );


In [ ]:

    drop_sql = f"drop view if exists {SCHEMA}.{PREFIX}_training_{i} CASCADE;"
    drop_statement.append(drop_sql)
    

scored_spoke_persuasion_1minus_20191209
scored_spoke_persuasion_1plus_20191209
scored_spoke_support_1box_20191209

ntiles_sql = "\n".join([",NTILE(100) OVER (ORDER BY {v_item}) AS {v_item}_100".format(v_item=k) for k in DV_LIST])

